In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
import numpy as np
from scipy.signal import wiener
from tqdm import tqdm
import pandas as pd
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim

sys.path.append(str(Path("..").resolve()))
from src import *

# Wiener Filter

In [3]:
astro_x_path = DATASETS["oabf_astro"].dir / "x.tiff"
astro_y_path = DATASETS["oabf_astro"].dir / "y.tiff"
astro_x, astro_y = (Recording(_, max_frames=frames_per_patch*2) for _ in [astro_x_path, astro_y_path])

In [ ]:
# Video sintetico: 20 frame 64x64 con un quadrato bianco che si muove
frames = 20
h, w = 64, 64
video = np.zeros((frames, h, w))
for t in range(frames):
    video[t, 16:32, 16+t:32+t] = 1.0

# Aggiungo rumore gaussiano
np.random.seed(0)
noisy = video + 0.2 * np.random.randn(*video.shape)

# Wiener frame-per-frame (2D)
denoised_2d = np.array([wiener(noisy[t], (5,5)) for t in range(frames)])

# Wiener 3D semplice: blocco di 3 frame (finestra temporale)
def wiener3d(noisy, win=(3,5,5)):
    T,H,W = noisy.shape
    out = np.zeros_like(noisy)
    pad = (win[0]//2, win[1]//2, win[2]//2)
    noisy_p = np.pad(noisy, [(pad[0],)]*2 + [(pad[1],)]*2 + [(pad[2],)]*2, mode="reflect")
    for t in range(T):
        for i in range(H):
            for j in range(W):
                block = noisy_p[t:t+win[0], i:i+win[1], j:j+win[2]]
                local_mean = np.mean(block)
                local_var = np.var(block)
                noise_var = 0.04  # ipotesi sigma_n^2 = 0.2^2
                out[t,i,j] = local_mean + max(0, local_var - noise_var)/max(local_var, noise_var) * (noisy[t,i,j] - local_mean)
    return out

denoised_3d = wiener3d(noisy)

# Mostro un frame
plt.subplot(1,3,1); plt.imshow(noisy[10], cmap="gray"); plt.title("Noisy")
plt.subplot(1,3,2); plt.imshow(denoised_2d[10], cmap="gray"); plt.title("2D Wiener")
plt.subplot(1,3,3); plt.imshow(denoised_3d[10], cmap="gray"); plt.title("3D Wiener")
plt.show()
